# Лабораторная работа 3. Распознавание многозначных чисел

Цель: Используя код из лабораторной работы № 2 обучить нейросеть распознавать рукописные многозначные числа.



In [1]:
pip install wget # Пакет wget для скачивания файлов из Интернета

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=a3775ffd3ba8fe2114f6c42cfdda4036ca4c0d89f4f360c941b37bfba78e5ca3
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
import wget
for index in range(10):
  as_str = str(index)
  as_str += ".bmp"
  wget.download('http://xyla.istu.webappz.ru/neuron/'+as_str)
wget.download(('http://xyla.istu.webappz.ru/neuron/mnist.pkl.gz'))
# база данных MNIST содержит 70 000 изображений рукописных цифр, разделенных на три набора:
# training_data – набор из 50 000 изображений предназначен для обучения нейронных сетей;
# validation_data – набор из 10 000 изображений предназначен для текущей оценки работы алгоритма обучения и подбора параметров обучения (используется в последующих лабораторных работах);
# test_data – набор из 10 000 изображений предназначен для проверки работы нейронной сетей.
# Каждый набор состоит из двух списков: списка изображений (в градациях серого) и соответствующего списка цифр в диапазоне от 0 до 9 Изображение представлено в виде одномерного numpy-массива размера 784 = 28 × 28 значений от 0 до 1, где 0 соответствует черному цвету пиксела, а 1 – белому.

'mnist.pkl.gz'

In [3]:
# В качестве функции активации для нейронов сети используется сигмоидальная функция, вычисляющая выходной сигнал искусственного нейрона
def sigmoid(z): # определение сигмоидальной функции активации
    return 1.0/(1.0+np.exp(-z))

In [4]:
# Данный метод осуществляет подсчет выходных сигналов нейронной сети при заданных входных сигналах.
# Параметр a является массивом n × 1, где n – количество нейронов входного слоя. Функция np.dot вычисляет произведение матриц.
# Для подсчета выходных значений нейронной сети, необходимо один раз вызвать метод feedforward, в результате чего выходные сигналы будут последовательно вычислены для всех слоев нейронной сети.
def feedforward(self, a):
    for b, w in zip(self.biases, self.weights):
        a = sigmoid(np.dot(w, a)+b)
    return a

In [5]:
# Для реализации механизма обучения создаваемой нейронной сети добавим метод SGD, который реализует стохастический градиентный спуск. Метод имеет следующие параметры:
# «Training_data» – обучающая выборка, состоящая из пар вида ( , ), где – вектор входных сигналов, а – ожидаемый вектор выходных сигналов;
# «epochs» – количество эпох обучения;
# «mini_batch_size» - размер подвыборки;
# «eta» - скорость обучения;
# «test_data» - (необязательный параметр); если данный аргумент не пуст, то программа после каждой эпохи обучения осуществляет оценку работы сети и показывает достигнутый прогресс.
def SGD(                                      # Стохастический градиентный спуск
          self                                # указатель на объект класса
        , training_data                       # обучающая выборка
        , epochs                              # количество эпох обучения
        , mini_batch_size                     # размер подвыборки
        , eta                                 # скорость обучения
        , test_data                           # тестирующая выборка
        ):
    test_data = list(test_data)               # создаем список объектов тестирующей выборки
    n_test = len(test_data)                   # вычисляем длину тестирующей выборки
    training_data = list(training_data)       # создаем список объектов обучающей выборки
    n = len(training_data)                    # вычисляем размер обучающей выборки
    for j in range(epochs):                   # цикл по эпохам
        random.shuffle(training_data)         # перемешиваем элементы обучающей выборки
        mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]   # создаем подвыборки
        for mini_batch in mini_batches:                                                             # цикл по подвыборкам
            self.update_mini_batch(mini_batch, eta)                                                 # один шаг градиентного спуска
        print ("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))                  # смотрим прогресс в обучении
        with open('biases.pkl', 'wb') as f:
            pickle.dump(self.biases, f)
        with open('weights.pkl', 'wb') as f:
            pickle.dump(self.weights, f)

In [8]:
# Метод update_mini_batch вычисляет градиенты для каждого прецедента в подвыборке, а затем соответствующим образом обновляет веса и смещения нейронной сети
def update_mini_batch(                  # Шаг градиентного спуска
                      self              # указатель на объект класса
                    , mini_batch        # подвыборка
                    , eta               # скорость обучения
                    ):
    nabla_b = [np.zeros(b.shape) for b in self.biases]      # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
    nabla_w = [np.zeros(w.shape) for w in self.weights]     # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)
    for x, y in mini_batch:
        delta_nabla_b, delta_nabla_w = self.backprop(x, y)  # послойно вычисляем градиенты dC/db и dC/dw для текущего прецедента (x, y)
        nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)] # суммируем градиенты dC/db для различных прецедентов текущей подвыборки
        nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)] # суммируем градиенты dC/dw для различных прецедентов текущей подвыборки
    self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)] # обновляем все веса w нейронной сети
    self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)] # обновляем все смещения b нейронной сети

In [9]:
def backprop(                     # Алгоритм обратного распространения
              self                # указатель на объект класса
            , x                   # вектор входных сигналов
            , y                   # ожидаемый вектор выходных сигналов
            ):
    nabla_b = [np.zeros(b.shape) for b in self.biases]      # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
    nabla_w = [np.zeros(w.shape) for w in self.weights]     # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)

    # определение переменных
    activation = x                                          # выходные сигналы слоя (первоначально соответствует выходным сигналам 1-го слоя или входным сигналам сети)
    activations = [x]                                       # список выходных сигналов по всем слоям (первоначально содержит только выходные сигналы 1-го слоя)
    zs = []                                                 # список активационных потенциалов по всем слоям (первоначально пуст)

    # прямое распространение
    for b, w in zip(self.biases, self.weights):
        z = np.dot(w, activation)+b                         # считаем активационные потенциалы текущего слоя
        zs.append(z)                                        # добавляем элемент (активационные потенциалы слоя) в конец списка
        activation = sigmoid(z)                             # считаем выходные сигналы текущего слоя, применяя сигмоидальную функцию активации к активационным потенциалам слоя
        activations.append(activation)                      # добавляем элемент (выходные сигналы слоя) в конец списка

    # обратное распространение
    delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])    # считаем меру влияния нейронов выходного слоя L на величину ошибки (BP1)
    nabla_b[-1] = delta                                                         # градиент dC/db для слоя L (BP3)
    nabla_w[-1] = np.dot(delta, activations[-2].transpose())                    # градиент dC/dw для слоя L (BP4)
    for l in range(2, self.num_layers):
        z = zs[-l]                                                              # активационные потенциалы l-го слоя (двигаемся "Положение EcoRobotics.docx"списку справа налево)
        sp = sigmoid_prime(z)                                                   # считаем сигмоидальную функцию от активационных потенциалов l-го слоя
        delta = np.dot(self.weights[-l+1].transpose(), delta) * sp              # считаем меру влияния нейронов l-го слоя на величину ошибки (BP2)
        nabla_b[-l] = delta                                                     # градиент dC/db для l-го слоя (BP3)
        nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())              # градиент dC/dw для l-го слоя (BP4)
    return (nabla_b, nabla_w)

In [10]:
# Метод возвращает количество прецедентов тестирующей выборки, для которых нейронная сеть выдает правильный результат
# Тестирующая выборка состоит из пар (x, y), где x – вектор размерности 784, содержащий изображение цифры, а y – целое числовое значение цифры, изображенной на картинке
# Ответ нейронной сети определяется как номер нейрона в выходном слое, имеющего наибольшее значение функции активации
# Метод evaluate вызывается в методе SGD после завершения очередной эпохи обучения
def evaluate(self, test_data): # Оценка прогресса в обучении
    test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
    return sum(int(x == y) for (x, y) in test_results)

In [11]:
# Вычисление частных производных стоимостной функции по выходным сигналам последнего слоя
def cost_derivative(self, output_activations, y): # Вычисление частных производных стоимостной функции по выходным сигналам последнего слоя
    return (output_activations-y)

In [12]:
def sigmoid_prime(z):# Производная сигмоидальной функции
    return sigmoid(z)*(1-sigmoid(z))

In [13]:
# Функции для работы с базой данных MNIST
import gzip           # библиотека для сжатия и распаковки файлов gzip и gunzip.
import pickle         # библиотека для сохранения и загрузки сложных объектов Python.
import numpy as np    # библиотека для работы с матрицами

def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb')                                             # открываем сжатый файл gzip в двоичном режиме
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')   # загружам таблицы из файла
    f.close()                                                                       # закрываем файл
    return (training_data, validation_data, test_data)

In [14]:
# Для преобразования числа в вектор-столбец (10-мерный numpy массив), используется следующая функция vectorized_result.
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1
    return e

In [15]:
# Для использования базы данных MNIST в нашей программе необходимо скорректировать форматы наборы training_data, validation_data и test_data
# Это делается в функции load_data_wrapper
def load_data_wrapper():
    tr_d, va_d, te_d = load_data()                                  # инициализация наборов данных в формате MNIST
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]    # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    training_results = [vectorized_result(y) for y in tr_d[1]]      # представление цифр от 0 до 9 в виде массивов размера 10 на 1
    training_data = zip(training_inputs, training_results)          # формируем набор обучающих данных из пар (x, y)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]  # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    validation_data = zip(validation_inputs, va_d[1])               # формируем набор данных проверки из пар (x, y)
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]        # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    test_data = zip(test_inputs, te_d[1])                           # формируем набор тестовых данных из пар (x, y)
    return (training_data, validation_data, test_data)

training_data, validation_data,test_data = load_data_wrapper()
# Данная функция преобразует training_data в список, содержащий 50 000 пар (x, y), где x является 784-мерным numpy-массивом, содержащим входное изображение
# а y – это 10-мерный numpy-массив, представляющий собой вектор, у которого координата с порядковым номером, соответствующим цифре на изображении
# равняется единице, а остальные координаты нулевые. Аналогичные преобразования делаются для наборов validation_data и test_data.

In [16]:
import numpy as np # библиотека функций для работы с матрицами
import pickle

def sigmoid(z): # определение сигмоидальной функции активации
    return 1.0/(1.0+np.exp(-z))

In [17]:
from IPython.lib.display import exists
"""""""""""""""""""""""""""""""""""""""
Модуль создания и обучения нейронной сети для распознавания рукописных цифр с использованием метода градиентного спуска.
Группа: АСУб-20-2
ФИО: Арбакова Анастасия Вячеславовна
"""""""""""""""""""""""""""""""""""""""
#### Библиотеки
# Стандартные библиотеки
import random               # библиотека функций для генерации случайных значений
import os.path

# Сторонние библиотеки
import numpy as np          # библиотека функций для работы с матрицами

""" ---Раздел описаний--- """
""" --Описание класса Network--"""
class Network(object):                    # используется для описания нейронной сети
    def __init__(self, sizes):            # конструктор класса, self – указатель на объект класса, sizes – список размеров слоев нейронной сети
        self.num_layers = len(sizes)      # задаем количество слоев нейронной сети
        self.sizes = sizes                # задаем список размеров слоев нейронной сети
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]                          # задаем случайные начальные смещения
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]     # задаем случайные начальные веса связей
        if(os.path.isfile('biases.pkl')):
          with open('biases.pkl', 'rb') as f:     #открытие файла со смещениями
            self.biases = pickle.load(f)
        if(os.path.isfile('weights.pkl')):
          with open('weights.pkl', 'rb') as f:    #открытие файла с весами
            self.weights = pickle.load(f)

    # В качестве функции активации для нейронов сети используется сигмоидальная функция, вычисляющая выходной сигнал искусственного нейрона
    def sigmoid(z):                               # определение сигмоидальной функции активации
        return 1.0/(1.0+np.exp(-z))
    def feedforward(self, a):                     # подсчет выходных сигналов нейронной сети при заданных входных сигналах
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    # Метод возвращает количество прецедентов тестирующей выборки, для которых нейронная сеть выдает правильный результат
    def evaluate(self, test_data):      # Оценка прогресса в обучении
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y): # Вычисление частных производных стоимостной функции по выходным сигналам последнего слоя
        return (output_activations-y)

    def backprop(           # Алгоритм обратного распространения
              self          # указатель на объект класса
            , x             # вектор входных сигналов
            , y             # ожидаемый вектор выходных сигналов
            ):
        nabla_b = [np.zeros(b.shape) for b in self.biases]        # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
        nabla_w = [np.zeros(w.shape) for w in self.weights]       # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)

        # определение переменных
        activation = x                                            # выходные сигналы слоя (первоначально соответствует выходным сигналам 1-го слоя или входным сигналам сети)
        activations = [x]                                         # список выходных сигналов по всем слоям (первоначально содержит только выходные сигналы 1-го слоя)
        zs = []                                                   # список активационных потенциалов по всем слоям (первоначально пуст)

        # прямое распространение
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b                           # считаем активационные потенциалы текущего слоя
            zs.append(z)                                          # добавляем элемент (активационные потенциалы слоя) в конец списка
            activation = sigmoid(z)                               # считаем выходные сигналы текущего слоя, применяя сигмоидальную функцию активации к активационным потенциалам слоя
            activations.append(activation)                        # добавляем элемент (выходные сигналы слоя) в конец списка

        # обратное распространение
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])      # считаем меру влияния нейронов выходного слоя L на величину ошибки (BP1)
        nabla_b[-1] = delta                                                           # градиент dC/db для слоя L (BP3)
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())                      # градиент dC/dw для слоя L (BP4)
        for l in range(2, self.num_layers):
            z = zs[-l]                                                                # активационные потенциалы l-го слоя (двигаемся "Положение EcoRobotics.docx"списку справа налево)
            sp = sigmoid_prime(z)                                                     # считаем сигмоидальную функцию от активационных потенциалов l-го слоя
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp                # считаем меру влияния нейронов l-го слоя на величину ошибки (BP2)
            nabla_b[-l] = delta                                                       # градиент dC/db для l-го слоя (BP3)
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())                # градиент dC/dw для l-го слоя (BP4)
        return (nabla_b, nabla_w)

    # Метод update_mini_batch вычисляет градиенты для каждого прецедента в подвыборке, а затем соответствующим образом обновляет веса и смещения нейронной сети
    def update_mini_batch(        # Шаг градиентного спуска
                      self        # указатель на объект класса
                    , mini_batch  # подвыборка
                    , eta         # скорость обучения
                    ):
        nabla_b = [np.zeros(b.shape) for b in self.biases]                            # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
        nabla_w = [np.zeros(w.shape) for w in self.weights]                           # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)                        # послойно вычисляем градиенты dC/db и dC/dw для текущего прецедента (x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]             # суммируем градиенты dC/db для различных прецедентов текущей подвыборки
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]             # суммируем градиенты dC/dw для различных прецедентов текущей подвыборки
        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)] # обновляем все веса w нейронной сети
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)] # обновляем все смещения b нейронной сети

# 1
    # Для реализации механизма обучения создаваемой нейронной сети добавим метод SGD, который реализует стохастический градиентный спуск
    def SGD(              # Стохастический градиентный спуск
          self            # указатель на объект класса
        , training_data   # обучающая выборка
        , epochs          # количество эпох обучения
        , mini_batch_size # размер подвыборки
        , eta             # скорость обучения
        , test_data       # тестирующая выборка
        ):
        test_data = list(test_data)         # создаем список объектов тестирующей выборки
        n_test = len(test_data)             # вычисляем длину тестирующей выборки
        training_data = list(training_data) # создаем список объектов обучающей выборки
        n = len(training_data)              # вычисляем размер обучающей выборки
        for j in range(epochs):             # цикл по эпохам
            random.shuffle(training_data)   # перемешиваем элементы обучающей выборки
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)] # создаем подвыборки
            for mini_batch in mini_batches: # цикл по подвыборкам
                self.update_mini_batch(mini_batch, eta) # один шаг градиентного спуска
            print ("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test)) # смотрим прогресс в обучении
            with open('biases.pkl', 'wb') as f:
                pickle.dump(self.biases, f)
            with open('weights.pkl', 'wb') as f:
                pickle.dump(self.weights, f)
        """ --Конец описания класса Network--"""
""" --- Конец раздела описаний--- """

""" ---Тело программы--- """
net = Network([2, 3, 1])                    # создаем нейронную сеть из трех слоев
""" ---Конец тела программы--- """
""" Вывод результата на экран: """
print('Сеть net:')
print('Количетво слоев:', net.num_layers)
for i in range(net.num_layers):
    print('Количество нейронов в слое', i,':',net.sizes[i])
for i in range(net.num_layers-1):
    print('W_',i+1,':')
    print(np.round(net.weights[i],2))
    print('b_',i+1,':')
    print(np.round(net.biases[i],2))

Сеть net:
Количетво слоев: 3
Количество нейронов в слое 0 : 2
Количество нейронов в слое 1 : 3
Количество нейронов в слое 2 : 1
W_ 1 :
[[ 0.82  0.65]
 [ 0.8  -0.37]
 [-0.41 -0.92]]
b_ 1 :
[[ 1.  ]
 [ 1.33]
 [-0.8 ]]
W_ 2 :
[[-1.15  0.73 -0.21]]
b_ 2 :
[[0.92]]


In [18]:
net = Network([784, 30, 10])
# Параметры, указанные при вызове данного метода, определяют топологию создаваемой сети
# Таким образом, в результате выполнения команды будет создана сеть, состоящая из трех слоев:
# входной слой сети состоит из 784-х нейронов
# внутренний слой из 30 нейронов
# выходной слой из 10 нейронов

In [19]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data) # процедура обучения созданной нейронной сети, включающая 30 эпох
# Параметры, указанные при вызове метода SGD:
# обучающая выборка
# количество эпох обучения
# размер подвыборки
# скорость обучения
# тестирующая выборка
# Обучение может занять несколько минут. В ходе обучения будет выдаваться информация о пройденных эпохах.
# Для каждой эпохи выводится отношение количества правильно распознанных цифр к общему количеству цифр в тестовой выборке.
# Например, запись Epoch 6: 9374 / 10000 говорит о том, что в результате
# эпохи обучения с номером 6 достигнута точность распознавания ≈ 0.94, что составляет 94%.

Epoch 0: 9062 / 10000
Epoch 1: 9285 / 10000
Epoch 2: 9303 / 10000
Epoch 3: 9369 / 10000
Epoch 4: 9394 / 10000
Epoch 5: 9418 / 10000
Epoch 6: 9405 / 10000
Epoch 7: 9459 / 10000
Epoch 8: 9443 / 10000
Epoch 9: 9464 / 10000
Epoch 10: 9458 / 10000
Epoch 11: 9473 / 10000
Epoch 12: 9500 / 10000
Epoch 13: 9494 / 10000
Epoch 14: 9519 / 10000
Epoch 15: 9492 / 10000
Epoch 16: 9512 / 10000
Epoch 17: 9513 / 10000
Epoch 18: 9505 / 10000
Epoch 19: 9530 / 10000
Epoch 20: 9520 / 10000
Epoch 21: 9531 / 10000
Epoch 22: 9521 / 10000
Epoch 23: 9508 / 10000
Epoch 24: 9520 / 10000
Epoch 25: 9520 / 10000
Epoch 26: 9523 / 10000
Epoch 27: 9525 / 10000
Epoch 28: 9531 / 10000
Epoch 29: 9535 / 10000


In [20]:
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

In [21]:
from PIL import Image
number =  input("Введите число: ")
num_arr = [int(a) for a in str(number)]
print (num_arr)
img = Image.open(str(num_arr[0])+'.bmp')
for index in range(len(num_arr)):
  if(index != 0):
    img = get_concat_h(img,Image.open(str(num_arr[index])+'.bmp'))
display(img)

Введите число: 56789
[5, 6, 7, 8, 9]


In [22]:
from PIL import Image
import numpy

for index in range(len(num_arr)):
    as_str = str(num_arr[index])
    as_str += ".bmp"
    im=Image.open(as_str) #открытие рисунка с 2-кой
    testArray=numpy.ndarray(shape=(784, 1), dtype=float, order='F') #создание контейнера для копирования значения пикселей изображения

    #операция копирования в контейнер пикселей изображения
    for i2 in range(28):
        for i1 in range(28):
            if (im.getpixel((i1, i2))) / 255 > 0:
                #вынужден был применять костыль, т.к. 1 в значении пикселей нейросеть неадекватно воспринимает
                testArray[i1 + i2 * 28, 0] = (im.getpixel((i1, i2))) / 255 - 0.001
            else:
                testArray[i1 + i2 * 28,0] = (im.getpixel((i1, i2))) / 255

    netSm = Network([784, 30, 10]) #создаем нейросеть с обученными весами и смещениями
    print(as_str)
    display(im)
    print(netSm.feedforward(testArray)) #распознаем изображение с помощью нейросети
    print(as_str)
    print()

5.bmp


[[3.00200886e-09]
 [6.09720659e-06]
 [3.16750981e-06]
 [3.69913075e-08]
 [3.97812473e-02]
 [4.21910483e-07]
 [2.07425563e-02]
 [1.57541749e-08]
 [1.02441385e-03]
 [9.49353098e-04]]
5.bmp

6.bmp


[[3.06865975e-04]
 [1.28713763e-06]
 [7.64518658e-06]
 [8.28956850e-12]
 [5.15276719e-09]
 [2.34980888e-01]
 [9.74784951e-01]
 [4.69640718e-12]
 [3.38070067e-03]
 [4.16153963e-05]]
6.bmp

7.bmp


[[5.32451826e-15]
 [1.78096139e-06]
 [5.44622456e-04]
 [7.12241360e-02]
 [8.55373129e-11]
 [4.19629101e-07]
 [1.07444768e-12]
 [9.24540232e-01]
 [4.75226203e-05]
 [3.19305034e-10]]
7.bmp

8.bmp


[[2.23591816e-07]
 [1.26645670e-05]
 [2.91580566e-08]
 [1.62868456e-03]
 [1.04599330e-06]
 [2.76838833e-07]
 [2.96288447e-10]
 [2.77256572e-05]
 [9.91266826e-01]
 [1.50227257e-01]]
8.bmp

9.bmp


[[2.55850590e-07]
 [3.57391902e-05]
 [1.95778728e-08]
 [7.46067633e-01]
 [7.98407361e-11]
 [1.46628156e-04]
 [1.73705848e-06]
 [1.46061185e-10]
 [7.96569247e-05]
 [7.90086539e-04]]
9.bmp

